In [ ]:
!pip install d2l

In [ ]:
#3.3.1. 生成数据集
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [ ]:
#3.3.2. 读取数据集
#将 features 和 labels 作为API的参数传递，并在实例化数据迭代器对象时指定 batch_size。此外，布尔值 is_train 表示是否希望数据迭代器对象在每个迭代周期内打乱数据。
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器。"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

In [ ]:
#3.3.3. 定义模型
#在 PyTorch 中，全连接层在 Linear 类中定义。值得注意的是，我们将两个参数传递到 nn.Linear 中。第一个指定输入特征形状，即 2，第二个指定输出特征形状，输出特征形状为单个标量，因此为 1。
# `nn` 是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [ ]:
#3.3.4. 初始化模型参数
#指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为零。
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

In [ ]:
#3.3.5. 定义损失函数
#计算均方误差使用的是MSELoss类，也称为平方  L2  范数。默认情况下，它返回所有样本损失的平均值。
loss = nn.MSELoss()

In [ ]:
#3.3.6. 定义优化算法
#小批量随机梯度下降算法是一种优化神经网络的标准工具，PyTorch 在 optim 模块中实现了该算法的许多变种。
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [ ]:
#3.3.7. 训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

In [ ]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)